In [1]:
#分析に必要なライブラリ、モジュールをimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as display
sns.set()
import urllib.request
from io import StringIO
#回帰分析で使うライブラリ
import scipy.stats as stats
import statsmodels.formula.api as smf
#scikit-Learnで使うもの
from sklearn.impute import SimpleImputer #欠損値の補間
from sklearn.model_selection import train_test_split #データの分割
from sklearn.preprocessing import MinMaxScaler #正規化
from sklearn.preprocessing import StandardScaler #標準化
from sklearn.linear_model import LinearRegression #重回帰モデル
from sklearn.linear_model import Ridge #　Ridge回帰
from sklearn.linear_model import Lasso #Lasso回帰
from sklearn.preprocessing import PolynomialFeatures #高次項の生成
from sklearn.metrics import r2_score #決定係数の計算
from sklearn.metrics import mean_squared_error #MSEの計算
from sklearn.metrics import mean_absolute_error #MAEの計算

In [2]:
%pip install xgboost

import xgboost as xgb

Note: you may need to restart the kernel to use updated packages.


In [28]:
%pip install lightgbm

import lightgbm as lgb


Note: you may need to restart the kernel to use updated packages.


In [3]:
#WEB上のcsvデータを読み込んでデータフレームとする。
import pandas as pd
#データを読み込む関数を定義
def read_csv(file_name):
    print(file_name)
    df = pd.read_csv(file_name)
    return df

#WEB上のデータのURLを指定
file_train = "smbc/train.csv"
file_test = "smbc/test.csv"
file_sample_submition = "smbc/sample_submission.csv"
#データを読み込む
df_train = read_csv(file_train)
df_test = read_csv(file_test)
df_sample_submission = read_csv(file_sample_submition)

#データの概要を確認
display.display(df_train.head())
display.display(df_test.head())
display.display(df_sample_submission.head())



smbc/train.csv
smbc/test.csv
smbc/sample_submission.csv


,Unnamed: 0,created_at,tree_dbh,curb_loc,health,steward,guards,sidewalk,user_type,problems,...,nta,nta_name,borocode,boro_ct,boroname,zip_city,cb_num,st_senate,st_assem,cncldist
0,0,2015-06-29,14,OnCurb,1,NaN,NaN,Damage,Volunteer,NaN,...,QN45,Douglas Manor-Douglaston-Little Neck,4,4152901,Queens,Little Neck,411,11,25,23
1,1,2016-09-21,5,OnCurb,1,3or4,Helpful,NoDamage,Volunteer,NaN,...,BX05,Bedford Park-Fordham North,2,2039901,Bronx,Bronx,207,33,78,15
2,2,2015-09-13,26,OnCurb,2,NaN,NaN,NoDamage,Volunteer,StonesBranchLights,...,SI01,Annadale-Huguenot-Prince's Bay-Eltingville,5,5017011,Staten Island,Staten Island,503,24,62,51
3,3,2016-05-09,15,OnCurb,0,NaN,NaN,Damage,NYC Parks Staff,NaN,...,SI11,Charleston-Richmond Valley-Tottenville,5,5024401,Staten Island,Staten Island,503,24,62,51
4,4,2016-06-24,23,OnCurb,1,NaN,NaN,NoDamage,Volunteer,Stones,...,MN03,Central Harlem North-Polo Grounds,1,1022102,Manhattan,New York,110,30,70,9


,Unnamed: 0,created_at,tree_dbh,curb_loc,steward,guards,sidewalk,user_type,problems,spc_common,...,nta,nta_name,borocode,boro_ct,boroname,zip_city,cb_num,st_senate,st_assem,cncldist
0,19984,2015-09-08,15,OnCurb,NaN,NaN,Damage,NYC Parks Staff,NaN,Callery pear,...,BK17,Sheepshead Bay-Gerritsen Beach-Manhattan Beach,3,3062000,Brooklyn,Brooklyn,315,22,45,48
1,19985,2015-10-15,5,OnCurb,1or2,NaN,NoDamage,Volunteer,NaN,cherry,...,BX62,Woodlawn-Wakefield,2,2045101,Bronx,Bronx,212,34,81,11
2,19986,2016-08-03,4,OnCurb,NaN,Unsure,NoDamage,Volunteer,NaN,littleleaf linden,...,QN60,Kew Gardens,4,4013800,Queens,Kew Gardens,409,14,27,29
3,19987,2015-08-06,7,OnCurb,NaN,NaN,NoDamage,TreesCount Staff,NaN,dawn redwood,...,BK09,Brooklyn Heights-Cobble Hill,3,3000301,Brooklyn,Brooklyn,302,26,52,33
4,19988,2015-10-21,6,OnCurb,1or2,Helpful,NoDamage,TreesCount Staff,Stones,purple-leaf plum,...,BK75,Bedford,3,3025100,Brooklyn,Brooklyn,303,25,56,36


,19984,1
0,19985,2
1,19986,0
2,19987,1
3,19988,2
4,19989,0


In [4]:
#df_trainのデータの概要を確認
display.display(df_train.describe())

,Unnamed: 0,tree_dbh,health,borocode,boro_ct,cb_num,st_senate,st_assem,cncldist
count,19984.000000,19984.000000,19984.000000,19984.000000,1.998400e+04,19984.000000,19984.000000,19984.000000,19984.000000
mean,9991.500000,11.694906,0.858036,3.360088,3.405346e+06,343.159077,21.111189,51.913981,30.346327
std,5769.028226,8.198150,0.437807,1.232246,1.239741e+06,122.264942,7.188537,18.642931,14.824068
min,0.000000,1.000000,0.000000,1.000000,1.000202e+06,101.000000,10.000000,23.000000,1.000000
25%,4995.750000,5.000000,1.000000,3.000000,3.005900e+06,301.000000,15.000000,33.000000,19.000000
50%,9991.500000,9.000000,1.000000,4.000000,4.009200e+06,402.000000,22.000000,56.000000,31.000000
75%,14987.250000,17.000000,1.000000,4.000000,4.122300e+06,413.000000,25.000000,64.000000,44.000000
max,19983.000000,42.000000,2.000000,5.000000,5.031902e+06,503.000000,36.000000,87.000000,51.000000


In [ ]:
#df_trainのヒストグラムを確認
df_train.hist(bins=50, figsize=(20,15))


In [5]:
#df_trainのデータの欠損値を確認
df_train.isnull().sum()

Unnamed: 0        0
created_at        0
tree_dbh          0
curb_loc          0
health            0
steward       14883
guards        14943
sidewalk          0
user_type         0
problems      12243
spc_common        0
spc_latin         0
nta               0
nta_name          0
borocode          0
boro_ct           0
boroname          0
zip_city          0
cb_num            0
st_senate         0
st_assem          0
cncldist          0
dtype: int64

In [23]:
#df_trainのcreated_atをdatetime型に変換
df_train["created_at"] = pd.to_datetime(df_train["created_at"])

print(df_train["created_at"].dtypes)

datetime64[ns]


In [17]:
#カテゴリ型への変換
# カテゴリ型への変換
for col in ['created_at', 'curb_loc', 'steward', 'guards', 'sidewalk', 'user_type', 'problems', 'spc_common', 'spc_latin', 'nta', 'nta_name', 'boroname', 'zip_city']:
    df_train[col] = df_train[col].astype('category')
    df_test[col] = df_test[col].astype('category')

#df_train,df_testの表示
display.display(df_train.tail())
display.display(df_test.tail(10))
display.display(df_test.head())

,Unnamed: 0,created_at,tree_dbh,curb_loc,health,steward,guards,sidewalk,user_type,problems,...,nta,nta_name,borocode,boro_ct,boroname,zip_city,cb_num,st_senate,st_assem,cncldist
19979,19979,2016-07-15,19,OnCurb,2,NaN,NaN,Damage,Volunteer,NaN,...,SI24,Todt Hill-Emerson Hill-Heartland Village-Light...,5,5018100,Staten Island,Staten Island,502,24,62,50
19980,19980,2016-07-08,5,OnCurb,1,NaN,NaN,NoDamage,NYC Parks Staff,NaN,...,SI32,Rossville-Woodrow,5,5020804,Staten Island,Staten Island,503,24,62,51
19981,19981,2015-08-20,21,OnCurb,0,NaN,NaN,Damage,Volunteer,NaN,...,SI48,Arden Heights,5,5017010,Staten Island,Staten Island,503,24,62,51
19982,19982,2016-06-20,4,OnCurb,1,1or2,NaN,NoDamage,NYC Parks Staff,NaN,...,MN25,Battery Park City-Lower Manhattan,1,1031704,Manhattan,New York,101,26,65,1
19983,19983,2015-08-19,31,OnCurb,1,NaN,NaN,Damage,NYC Parks Staff,Stones,...,BX59,Westchester-Unionport,2,2007800,Bronx,Bronx,209,32,87,18


,Unnamed: 0,created_at,tree_dbh,curb_loc,steward,guards,sidewalk,user_type,problems,spc_common,...,nta,nta_name,borocode,boro_ct,boroname,zip_city,cb_num,st_senate,st_assem,cncldist
19692,39958,2015-08-18,18,OnCurb,NaN,NaN,NoDamage,Volunteer,Stones,Sophora,...,BK58,Flatlands,3,3067800,Brooklyn,Brooklyn,318,21,59,46
19693,39959,2015-07-18,5,OnCurb,1or2,Harmful,NoDamage,Volunteer,NaN,honeylocust,...,QN07,Hollis,4,4050000,Queens,Hollis,412,14,29,27
19694,39960,2015-08-09,20,OnCurb,NaN,NaN,NoDamage,TreesCount Staff,NaN,sawtooth oak,...,QN61,Jamaica,4,4046000,Queens,Jamaica,412,11,29,27
19695,39961,2015-10-23,13,OnCurb,NaN,NaN,NoDamage,TreesCount Staff,NaN,cherry,...,BK45,Georgetown-Marine Park-Bergen Beach-Mill Basin,3,3070600,Brooklyn,Brooklyn,318,19,59,46
19696,39962,2015-09-05,14,OnCurb,NaN,NaN,Damage,Volunteer,NaN,English oak,...,SI28,Port Richmond,5,5024700,Staten Island,Staten Island,501,23,61,49
19697,39964,2015-09-10,3,OnCurb,1or2,Harmful,Damage,Volunteer,StonesRootOther,Japanese snowbell,...,SI45,New Dorp-Midland Beach,5,5012200,Staten Island,Staten Island,502,24,64,50
19698,39965,2016-09-09,9,OnCurb,1or2,NaN,NoDamage,NYC Parks Staff,NaN,silver maple,...,QN23,College Point,4,4093900,Queens,College Point,407,11,27,19
19699,39966,2015-12-31,16,OnCurb,NaN,NaN,Damage,Volunteer,NaN,Callery pear,...,BX01,Claremont-Bathgate,2,2039500,Bronx,Bronx,206,33,86,15
19700,39967,2016-09-07,24,OnCurb,NaN,NaN,Damage,TreesCount Staff,RootOtherTrunkOtherBranchOther,London planetree,...,BK79,Ocean Hill,3,3037300,Brooklyn,Brooklyn,316,25,55,41
19701,39968,2016-09-12,25,OnCurb,NaN,NaN,NoDamage,TreesCount Staff,Stones,London planetree,...,SI24,Todt Hill-Emerson Hill-Heartland Village-Light...,5,5017700,Staten Island,Staten Island,502,24,63,50


,Unnamed: 0,created_at,tree_dbh,curb_loc,steward,guards,sidewalk,user_type,problems,spc_common,...,nta,nta_name,borocode,boro_ct,boroname,zip_city,cb_num,st_senate,st_assem,cncldist
0,19984,2015-09-08,15,OnCurb,NaN,NaN,Damage,NYC Parks Staff,NaN,Callery pear,...,BK17,Sheepshead Bay-Gerritsen Beach-Manhattan Beach,3,3062000,Brooklyn,Brooklyn,315,22,45,48
1,19985,2015-10-15,5,OnCurb,1or2,NaN,NoDamage,Volunteer,NaN,cherry,...,BX62,Woodlawn-Wakefield,2,2045101,Bronx,Bronx,212,34,81,11
2,19986,2016-08-03,4,OnCurb,NaN,Unsure,NoDamage,Volunteer,NaN,littleleaf linden,...,QN60,Kew Gardens,4,4013800,Queens,Kew Gardens,409,14,27,29
3,19987,2015-08-06,7,OnCurb,NaN,NaN,NoDamage,TreesCount Staff,NaN,dawn redwood,...,BK09,Brooklyn Heights-Cobble Hill,3,3000301,Brooklyn,Brooklyn,302,26,52,33
4,19988,2015-10-21,6,OnCurb,1or2,Helpful,NoDamage,TreesCount Staff,Stones,purple-leaf plum,...,BK75,Bedford,3,3025100,Brooklyn,Brooklyn,303,25,56,36


In [5]:
#df_trainを学習用データと検証用データに分割する
X_train, X_valid, y_train, y_valid = train_test_split(df_train.drop('health',axis=1), df_train['health'], test_size=0.2, random_state=0)

In [7]:
from sklearn.metrics import f1_score

#特徴量と目的変数をxgboostのデータ構造に変換する
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dvalid = xgb.DMatrix(X_valid, label=y_valid, enable_categorical=True)

#パラメータを指定してGBDT
params = { 'max_depth': 3, 'eta': 1, 'objective': 'multi:softmax', 'num_class': 3}

#モデルを学習する
num_round = 50
bst = xgb.train(params, dtrain, num_round)

#予測する
tr_pred = bst.predict(dvalid)

#予測精度を計算する
val_f1 = f1_score(y_valid, tr_pred, average='macro')
print(f"Validation F1 Score (Macro): {val_f1}")

Validation F1 Score (Macro): 0.3069724790974505


In [ ]:
pred = bst.predict(df_test)
sample_submit[1] = pred
sample_submit.to_csv('/content/drive/MyDrive/SMFG/submit.csv', header=None)

In [21]:
# df_testをxgboostのデータ構造に変換する
dtest = xgb.DMatrix(df_test, enable_categorical=True)
# 予測
pred = bst.predict(dtest)

# 提出用ファイルの作成
df_sample_submission[1] = pred

df_sample_submission.to_csv('submit.csv', header=None, index=False)
df_sample_submission.head()

ValueError: Length of values (19702) does not match length of index (19701)

In [17]:
#特徴量と目的変数をlightbgmのデータ構造に変換する
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

#ハイパーパラメータの設定
params = {
        'objective': 'regression',
        'num_leaves': 50,
        'seed': 71,
        'verbose': 0,
        'metrics':'rmse'}
num_round = 100

#学習の実行
model = lgb.train(params, lgb_train, num_boost_round=num_round,
                  valid_names=['train', 'valid'], valid_sets=[lgb_train, lgb_eval])

#バリデーションデータでのスコアの確認
va_pred = model.predict(X_valid)
mae = mean_absolute_error(y_valid, va_pred)
rmse = np.sqrt(mean_squared_error(y_valid, va_pred))
r2 = r2_score(y_valid, va_pred)
print(f'mae: {mae:.4f}')
print(f'rmse: {rmse:.4f}')
print(f'r2: {r2:.4f}')

#予測
y_pred = model.predict(df_test)


ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: created_at: object, curb_loc: object, steward: object, guards: object, sidewalk: object, user_type: object, problems: object, spc_common: object, spc_latin: object, nta: object, nta_name: object, boroname: object, zip_city: object